In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os

In [3]:
subject_data = np.genfromtxt('/Users/linggeli/cnn_graph/data/dat/SUB2001_cup_preproc.csv', delimiter=',')

In [4]:
subject_data.shape

(100981, 2)

In [5]:
subject_data[:, 1]

array([       nan,        nan, 1.27634868, ..., 0.8986297 , 0.09553765,
              nan])

In [6]:
np.sum(np.isnan(subject_data[:, 1]))

749

In [23]:
def load_time_series_new(new_data_dir, subject_id):
    subject_data = np.genfromtxt(os.path.join(new_data_dir, 'SUB{}_cup_preproc.csv'.format(subject_id)), delimiter=',')
    time_series_data = np.zeros((375, 290))
    for i in range(375):
        select = subject_data[:, 0] == i
        current_time_series = subject_data[select, 1]
        n_time = current_time_series.shape[0]
        if n_time > 0 and n_time < 270:
            time_series_data[i, 20:(20 + n_time)] = current_time_series
        elif n_time > 270:
            time_series_data[i, 20:] = current_time_series[:270]
        else:
            continue
    time_series_data[np.isnan(time_series_data)] = 0
    return time_series_data

In [24]:
new_data_dir = '/Users/linggeli/cnn_graph/data/dat'

In [25]:
td = load_time_series_new('/Users/linggeli/cnn_graph/data/dat/', 2011)

In [26]:
td.shape

(375, 290)

In [27]:
np.sum(np.isnan(td))

0

In [28]:
def get_subject_id_new(new_data_dir):
    new_data_files = os.listdir(new_data_dir)
    subject_id_list = [int(x[3:7]) for x in new_data_files if x[-3:] == 'csv']
    return subject_id_list

In [38]:
new_subject_id_list = get_subject_id_new(new_data_dir)
print(new_subject_id_list)

[3139, 3766, 2308, 2224, 2219, 2183, 2311, 2184, 2207, 3726, 2240, 2208, 2319, 2258, 3153, 3005, 3808, 2082, 2024, 2151, 3030, 3080, 2182, 2507, 2011, 2163, 2267, 2061, 2268, 2130, 2228, 2126, 2505, 2161, 2013, 2227, 2145, 3423, 2244, 2199, 2230, 3074, 2321, 2280, 2012, 2068, 2314, 2167, 2291, 2302]


In [34]:
def get_all_features_new(new_data_dir, behavioral_dir, subject_id_list, **kwargs):
    all_features = []
    for subject_id in tqdm(subject_id_list):
        time_series_data = load_time_series_new(new_data_dir, subject_id)
        behavioral_data = load_behavioral(behavioral_dir, 2001)  # HACK: all behavioral data are assumed to be the same
        time_series_data = preprocess_time_series(time_series_data)
        features = create_features(time_series_data, behavioral_data, **kwargs)
        all_features.append(features)
    return np.concatenate(all_features)

In [35]:
from data_utils import *

In [74]:
time_series_dir = '/Users/linggeli/cnn_graph/data/time_series/cup/'
behavioral_dir = '/Users/linggeli/cnn_graph/data/fMRIbehav/cup/'

In [87]:
new_X = get_all_features_new(new_data_dir, behavioral_dir, new_subject_id_list, n_time=64, coef_idx=range(1, 17))

100%|██████████| 50/50 [00:39<00:00,  1.26it/s]


In [88]:
subject_id_list = get_subject_id(time_series_dir)

In [89]:
current_X = get_all_features(time_series_dir, behavioral_dir, subject_id_list, n_time=64, coef_idx=range(1, 17))

100%|██████████| 209/209 [00:31<00:00,  6.29it/s]


In [90]:
print(new_X.shape)
print(current_X.shape)

(200, 375, 16)
(836, 375, 16)


In [91]:
X = np.concatenate([new_X, current_X], axis=0)

In [92]:
unit_y = (get_all_labels(behavioral_dir, [2001]) - 1).astype(int)
y = np.tile(unit_y, int((new_X.shape[0] + current_X.shape[0]) / 4))

In [93]:
print(X.shape)
print(y.shape)

(1036, 375, 16)
(1036,)


In [94]:
brain_regions = np.genfromtxt('/Users/linggeli/cnn_graph/data/brain_regions.csv', dtype=int, delimiter=',')[:40] - 1

In [95]:
X = X[:, brain_regions, :]
print(X.shape)

(1036, 40, 16)


In [96]:
np.save('X_259sub_40reg_16coef.npy', X)
np.save('y_259sub.npy', y)